In [ ]:
import pandas as pd
from datasets import load_dataset
import numpy as np
import torch.nn.functional as F

In [ ]:

dataset = load_dataset('ImruQays/Quran-Classical-Arabic-English-Parallel-texts')['train'].to_pandas()
dataset.drop(columns=dataset.columns[0], inplace=True)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:

def get_embedding(sentence):

    with torch.no_grad():
        tokens = tokenizer(sentence, return_tensors="pt")
        outputs = model(**tokens)
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        return sentence_embedding

In [ ]:
import numpy as np

def get_similarity(embedding1, embedding2):
    dot = np.dot(embedding1, embedding2)
    
    similarty = dot / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    
    return similarty

In [ ]:
get_similarity(get_embedding("Where is my balls?"), get_embedding("أين محمد"))

0.17989026

In [ ]:
test_subject = dataset.iloc[1400, [0, 7]].values
test_subject

array(['أَم يَقولونَ افتَراهُ قُل فَأتوا بِسورَةٍ مِثلِهِ وَادعوا مَنِ استَطَعتُم مِن دونِ اللَّهِ إِن كُنتُم صادِقينَ',
       'Or do they say, “He has forged it”? Say, “Then produce a single chapter like it, and call upon whomever you can, apart from God, if you are truthful.”'],
      dtype=object)

In [ ]:
import re
import regex  # Need regex module for full Unicode support

def remove_arabic_diacritics(text):
    # Define a regex pattern to match Arabic diacritics (Tashkeel)
    diacritics_pattern = regex.compile(r'[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652\u0653\u0654\u0655\u0656\u0657\u0658\u0659\u065A\u065B\u065C\u065D\u065E\u065F\u0670]')
    
    # Remove diacritics from the text using regex substitution
    text_clean = diacritics_pattern.sub('', text)
    
    return text_clean

# Example usage
clean_arabic_text = remove_arabic_diacritics(test_subject[0])
print(clean_arabic_text)

أم يقولون افتراه قل فأتوا بسورة مثله وادعوا من استطعتم من دون الله إن كنتم صادقين


In [ ]:
arabic_sentence, english_sentence = test_subject

In [ ]:
eng_sentence_part = english_sentence.split(".")[1]

In [ ]:
eng_sentence_part

' They advocate evil, and prohibit righteousness, and withhold their hands'

In [ ]:
eng_sentence_part
en_part_emb = get_embedding(eng_sentence_part)

In [ ]:
ar_sentence_part = ...

In [ ]:
scores = []

for word_index in range(5, len(arabic_sentence.split(" ")) + 1):
    ar_sentence_part = " ".join(arabic_sentence.split(" ")[5:word_index])
    ar_part_emb = get_embedding(ar_sentence_part)
    similarity_to_eng_part = get_similarity(en_part_emb, ar_part_emb)
    
    scores.append(similarity_to_eng_part)
    print()
    print(ar_sentence_part)
    print(f"{similarity_to_eng_part:.2%}")



7.07%

يَأمُرونَ
22.14%

يَأمُرونَ بِالمُنكَرِ
36.11%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ
43.72%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ
48.06%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ
44.85%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ
49.33%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم
53.00%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا
52.90%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا اللَّهَ
54.63%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا اللَّهَ فَنَسِيَهُم
52.76%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا اللَّهَ فَنَسِيَهُم إِنَّ
51.91%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا اللَّهَ فَنَسِيَهُم إِنَّ المُنافِقينَ
55.39%

يَأمُرونَ بِالمُنكَرِ وَيَنهَونَ عَنِ المَعروفِ وَيَقبِضونَ أَيدِيَهُم نَسُوا اللَّهَ فَنَسِيَهُم إِنَّ المُنافِقينَ هُمُ
50.07%



17

In [ ]:
arabic_sentence.split()[5:np.argmax(scores) + 5]

['يَأمُرونَ',
 'بِالمُنكَرِ',
 'وَيَنهَونَ',
 'عَنِ',
 'المَعروفِ',
 'وَيَقبِضونَ',
 'أَيدِيَهُم',
 'نَسُوا',
 'اللَّهَ',
 'فَنَسِيَهُم',
 'إِنَّ',
 'المُنافِقينَ']